In [2]:
"""
Task 8: Tool Calling & Creation
- Create multiple tools
- Bind tools methods

LLM: Ollama (llama3)
"""

from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate


# -------------------------------------------------
# Create Local LLM
# -------------------------------------------------
llm = ChatOllama(
    model="llama3",
    temperature=0
)


# =================================================
# DEFINE TOOLS
# =================================================
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b


@tool
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


@tool
def get_weather(city: str) -> str:
    """Get weather information for a city."""
    return f"The weather in {city} is cold."


tools = {
    "add_numbers": add_numbers,
    "multiply_numbers": multiply_numbers,
    "get_weather": get_weather,
}


# =================================================
# PROMPT: TOOL SELECTION 
# =================================================
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an assistant. Decide which tool to use.\n"
            "Available tools:\n"
            "- add_numbers(a, b)\n"
            "- multiply_numbers(a, b)\n"
            "- get_weather(city)\n\n"
            "Respond ONLY in this format:\n"
            "tool_name | arguments"
        ),
        ("human", "{input}")
    ]
)

chain = prompt | llm


# =================================================
# TOOL EXECUTION LOGIC
# =================================================
def execute_tool(tool_decision: str):
    tool_name, args = tool_decision.split("|")
    tool_name = tool_name.strip()
    args = args.strip()

    tool = tools.get(tool_name)

    if tool_name in ["add_numbers", "multiply_numbers"]:
        a, b = map(int, args.split(","))
        return tool.invoke({"a": a, "b": b})

    if tool_name == "get_weather":
        return tool.invoke({"city": args})

    return "Unknown tool"


# =================================================
# RUN EXAMPLES
# =================================================
def run_examples():
    queries = [
        "Add 10 and 20",
        "Multiply 7 and 6",
        "What is the weather in Delhi?"
    ]

    for query in queries:
        print(f"\nUser Query: {query}")

        decision = chain.invoke({"input": query}).content
        print("Tool Decision:", decision)

        result = execute_tool(decision)
        print("Tool Result:", result)


# -------------------------------------------------
# Main Execution
# -------------------------------------------------
if __name__ == "__main__":
    run_examples()




User Query: Add 10 and 20
Tool Decision: add_numbers | 10, 20
Tool Result: 30

User Query: Multiply 7 and 6
Tool Decision: multiply_numbers | 7, 6
Tool Result: 42

User Query: What is the weather in Delhi?
Tool Decision: get_weather | Delhi
Tool Result: The weather in Delhi is cold.
